In [1]:
from Bio.PDB import PDBParser,DSSP

from vispy import gloo, app, visuals

from vispy.util.transforms import perspective, translate

import numpy as np

from molecular_data import crgbaDSSP, restype, colorrgba, vrad, resdict

In [2]:
W,H = 1200, 800

vertex = """
#version 120

uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform vec3 u_light_position;
uniform vec3 u_light_spec_position;

attribute vec3  a_position;
attribute vec3  a_color;
attribute float a_radius;

varying vec3  v_color;
varying vec4  v_eye_position;
varying float v_radius;
varying vec3  v_light_direction;

void main (void) {
    v_radius = a_radius;
    v_color = a_color;

    v_eye_position = u_view * u_model * vec4(a_position,1.0);
    v_light_direction = normalize(u_light_position);
    float dist = length(v_eye_position.xyz);

    gl_Position = u_projection * v_eye_position;

    vec4  proj_corner = u_projection * vec4(a_radius, a_radius, v_eye_position.z, v_eye_position.w);  // # noqa
    gl_PointSize = 512.0 * proj_corner.x / proj_corner.w;
}
"""

fragment = """
#version 120

uniform mat4 u_model;
uniform mat4 u_view;
uniform mat4 u_projection;
uniform vec3 u_light_position;
uniform vec3 u_light_spec_position;

varying vec3  v_color;
varying vec4  v_eye_position;
varying float v_radius;
varying vec3  v_light_direction;
void main()
{
    vec2 texcoord = gl_PointCoord* 2.0 - vec2(1.0);
    float x = texcoord.x;
    float y = texcoord.y;
    float d = 1.0 - x*x - y*y;
    if (d <= 0.0)
        discard;

    float z = sqrt(d);
    vec4 pos = v_eye_position;
    pos.z += v_radius*z;
    vec3 pos2 = pos.xyz;
    pos = u_projection * pos;
    // gl_FragDepth = 0.5*(pos.z / pos.w)+0.5;
    vec3 normal = vec3(x,y,z);
    float diffuse = clamp(dot(normal, v_light_direction), 0.0, 1.0);

    // Specular lighting.
    vec3 M = pos2.xyz;
    vec3 O = v_eye_position.xyz;
    vec3 L = u_light_spec_position;
    vec3 K = normalize(normalize(L - M) + normalize(O - M));
    // WARNING: abs() is necessary, otherwise weird bugs may appear with some
    // GPU drivers...
    float specular = clamp(pow(abs(dot(normal, K)), 40.), 0.0, 1.0);
    vec3 v_light = vec3(1., 1., 1.);
    gl_FragColor.rgb = (.15*v_color + .55*diffuse * v_color
                        + .35*specular * v_light);
}
"""


#Function to define the center of the point cloud
def centroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return sum_x/length, sum_y/length, sum_z/length

class Canvas(app.Canvas):
    
    visualization_modes = ['cpk','backbone','aminoacid', 'dssp']
    
    def __init__(self, pdbdata, mode='cpk'):
        
        #Startup
        app.Canvas.__init__(self, keys='interactive', size =(W,H))
        
        #Loading shaders
        self.program = gloo.Program(vertex, fragment)
        
        #Analyze pdb file
        self.parser = PDBParser(QUIET=True, PERMISSIVE=True)
        self.structure = self.parser.get_structure('model',pdbdata)
        
        #DSSP prediction
        self.pmodel = self.structure[0]
        self.dssp = DSSP(self.pmodel, pdbdata)
        
        #Mode selection
        if mode not in Canvas.visualization_modes:
            raise Exception('Not recognized visualization mode %s' % mode)
        self.mode = mode
        
        #Camera settings
        self.translate = 120
        self.view = translate((0,0, -self.translate), dtype=np.float32)
        self.model = np.eye(4, dtype=np.float32)
        self.projection = perspective(45.0, self.size[0] / float(self.size[1]), 1.0, 1000.0)
        
        self.program['u_model'] = self.model
        self.program['u_view'] = self.view
        self.program['u_projection'] = self.projection
        
        #Load data depending on the mdoe
        self.atom_information()
        self.load_data()
        self.show()
        
        
        
    def atom_information(self):
        
        """Determines the coordinates, colors and sizes of the atoms depending on the mode"""
        
        if self.mode == 'cpk':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms()]
            self.natoms = len(self.atoms)
            #atom coordinates
            self.coordinates = np.array([atom.coord for atom in self.atoms])
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = [colorrgba(atom.get_id()) for atom in self.atoms]
            #atom radius
            self.radius = [vrad(atom.get_id()) for atom in self.atoms]
            
        elif self.mode == 'aminoacid':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms() if atom.get_parent().resname != 'HOH']
            self.natoms = len(self.atoms)
            #atom coordinates
            self.coordinates = np.array([atom.coord for atom in self.atoms])
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = [colorrgba(restype(atom.get_parent().resname)) for atom in self.atoms]
            #atom radius
            self.radius = [vrad(atom.get_id()) for atom in self.atoms]
            
        elif self.mode == 'backbone':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']
            self.natoms = len(self.atoms)
            #atom coordinates
            self.coordinates = np.array([atom.coord for atom in self.atoms])
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.color = []
            self.chains = []
            for chain in self.structure.get_chains():
                self.chains.append(chain)
                self.chain_length = len([atom for atom in chain.get_atoms() if atom.get_name() =='CA' or atom.get_name() =='N'])
                self.chain_color = np.append(np.random.rand(1,3),[1.0])
                self.color.append(np.tile(self.chain_color,(self.chain_length,1)))
            if len(self.chains)>1:
                self.color = np.concatenate(self.color)
            #atom radius
            self.radius = [vrad(atom.get_id()) for atom in self.atoms]
            
        elif self.mode == 'dssp':
            #list of atoms
            self.atoms = [atom for atom in self.structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']
            self.natoms = len(self.atoms)
            #atom coordinates
            self.coordinates = np.array([atom.coord for atom in self.atoms])
            self.center = centroid(self.coordinates)
            self.coordinates -= self.center
            #atom color
            self.struct3 = [self.dssp[key][2] for key in list(self.dssp.keys())]
            self.residues = [residue for residue in self.structure.get_residues() if residue.get_resname() in resdict.keys()]
            self.color = []
            for i in range(len(self.struct3)):
                self.dsspcolor = crgbaDSSP(self.struct3[i])
                self.n_atoms = len([atom for atom in self.residues[i] if atom.get_name() =='CA' or atom.get_name() == 'N'])
                self.color.append(np.tile(self.dsspcolor,(self.n_atoms,1)))
            if len(self.struct3)>1:
                self.color = np.concatenate(self.color)
            #atom radius
            self.radius = [vrad(atom.get_id()) for atom in self.atoms]
        
    def load_data(self):
        
        """Make an array with all the data and load it into VisPy Gloo"""
        
        data = np.zeros(self.natoms, [('a_position', np.float32, 3),
                            ('a_color', np.float32, 4),
                            ('a_radius', np.float32, 1)])

        data['a_position'] = self.coordinates
        data['a_color'] = self.color
        data['a_radius'] = self.radius #*self.pixel_scale

        self.program.bind(gloo.VertexBuffer(data))
        print 'Data loaded'

    
    def on_resize(self, event):
        gloo.set_viewport(0, 0, event.physical_size[0], event.physical_size[1])
        self.projection = perspective(45.0, event.size[0] / float(event.size[1]), 1.0, 1000.0)
        self.program['u_projection'] = self.projection
    
    def on_draw(self,event):
        gloo.clear()
        self.program.draw('points')

In [3]:
pdbdata = 'data/1yd9.pdb'
mvc = Canvas(pdbdata, mode='backbone')

Data loaded


In [4]:
mvc.coordinates

array([[  0.31410497,  16.20023   ,  -6.0729146 ],
       [ -0.8898974 ,  17.05423   ,  -5.875914  ],
       [  0.51610464,  18.566229  ,  -4.6129136 ],
       ...,
       [ 18.436104  , -18.714771  , -11.773915  ],
       [ 17.621103  , -20.100773  , -13.597915  ],
       [ 17.767103  , -21.054771  , -14.691915  ]], dtype=float32)

In [4]:
residues = [atom.get_parent().resname for atom in mvc.atoms if atom.get_parent().resname != 'HOH']
print residues

['GLY', 'GLY', 'PHE', 'PHE', 'THR', 'THR', 'VAL', 'VAL', 'LEU', 'LEU', 'SER', 'SER', 'THR', 'THR', 'LYS', 'LYS', 'SER', 'SER', 'LEU', 'LEU', 'PHE', 'PHE', 'LEU', 'LEU', 'GLY', 'GLY', 'GLN', 'GLN', 'LYS', 'LYS', 'LEU', 'LEU', 'GLN', 'GLN', 'VAL', 'VAL', 'VAL', 'VAL', 'GLN', 'GLN', 'ALA', 'ALA', 'ASP', 'ASP', 'ILE', 'ILE', 'ALA', 'ALA', 'SER', 'SER', 'ILE', 'ILE', 'ASP', 'ASP', 'SER', 'SER', 'ASP', 'ASP', 'ALA', 'ALA', 'VAL', 'VAL', 'VAL', 'VAL', 'HIS', 'HIS', 'PRO', 'PRO', 'THR', 'THR', 'ASN', 'ASN', 'THR', 'THR', 'ASP', 'ASP', 'PHE', 'PHE', 'TYR', 'TYR', 'ILE', 'ILE', 'GLY', 'GLY', 'GLY', 'GLY', 'GLU', 'GLU', 'VAL', 'VAL', 'GLY', 'GLY', 'SER', 'SER', 'THR', 'THR', 'LEU', 'LEU', 'GLU', 'GLU', 'LYS', 'LYS', 'LYS', 'LYS', 'GLY', 'GLY', 'GLY', 'GLY', 'LYS', 'LYS', 'GLU', 'GLU', 'PHE', 'PHE', 'VAL', 'VAL', 'GLU', 'GLU', 'ALA', 'ALA', 'VAL', 'VAL', 'LEU', 'LEU', 'GLU', 'GLU', 'LEU', 'LEU', 'ARG', 'ARG', 'LYS', 'LYS', 'LYS', 'LYS', 'ASN', 'ASN', 'GLY', 'GLY', 'PRO', 'PRO', 'LEU', 'LEU', 'GLU'

In [5]:
def centroid(arr):
    length = arr.shape[0]
    sum_x = np.sum(arr[:, 0])
    sum_y = np.sum(arr[:, 1])
    sum_z = np.sum(arr[:, 2])
    return sum_x/length, sum_y/length, sum_z/length

pdbdata = 'data/1yd9.pdb'

#Analyze pdb file
parser = PDBParser(QUIET=True, PERMISSIVE=True)
structure = parser.get_structure('model',pdbdata)

#DSSP prediction
pmodel = structure[0]
dssp = DSSP(pmodel, pdbdata)


atoms = [atom for atom in structure.get_atoms() if atom.get_name() == 'CA' or atom.get_name() == 'N']
natoms = len(atoms)

#atom coordinates
coordinates = np.array([atom.coord for atom in atoms])
center = centroid(coordinates)
coordinates -= center

#atom color
struct3 = [dssp[key][2] for key in list(dssp.keys())]
residues = [residue for residue in structure.get_residues() if residue.get_resname() in resdict.keys()]
color = []
for i in range(len(struct3)):
    dsspcolor = crgbaDSSP(struct3[i])
    n_atoms = len([atom for atom in residues[i] if atom.get_name() =='CA' or atom.get_name() == 'N'])
    color.append(np.tile(dsspcolor,(n_atoms,1)))
if len(struct3)>1:
    color = np.array(np.concatenate(color))
#atom radius
radius = [vrad(atom.get_id()) for atom in atoms]

In [6]:
natoms

1500

In [7]:
range(len(mvc.chains))


[0, 1, 2, 3]